In [ ]:
# Importing relevant libraries
import pandas as pd

In [ ]:
df1 = pd.read_excel('SDG2000_Yahoo_Financial.xlsx') # Dataset containing financial metrics taken from the API
df2 = pd.read_excel('SDG2000_Yahoo_Sustainability.xlsx') # Dataset containing sustainability metrics taken from the API
df3 = pd.read_excel('SDG2000-shareable-list_28.04.2022 (1).xlsx',sheet_name = '2. SDG2000') # Base dataset
df4 = pd.read_excel('SDG2000_Yahoo_Tickers.xlsx') # Dataset with tickers scraped from Yahoo Finanace
df5 = pd.read_excel('SDG2000_MSCI_Refinitive.xlsx') # Dataset with information scraped from MSCI and Refinitive

In [ ]:
df6 = df3.merge(df4, how = 'left') # Merging the base dataset with the tickers

In [ ]:
df7 = df6.merge(df1, how = 'left', left_on = 'Ticker', right_on = 'index') # Merging previous dataset with financial metrics

In [ ]:
df8 = df7.merge(df2, how = 'left', left_on = 'Ticker', right_on = 'company_ticker') # Merging previous dataset with sustainability metrics

In [ ]:
df9 = df5[['Name', 'Refinitive_Rating', 'MSCI_Rating']] # Selecting only relevant columns

In [ ]:
df = df8.merge(df9, how = 'left', left_on = 'Name', right_on ='Name') # Merging previous dataset with MSCI Refinitive

In [ ]:
# Adding 0s where the yahoo finance esg ratings are missing and reversing the scores, as in yahoo finance a lower score indicates better esg performance
df['totalEsg'] = df['totalEsg'].fillna(0)
df['totalEsg'] = 100 - df['totalEsg'] 
df.loc[df['totalEsg']== 100, 'totalEsg'] = 0

In [ ]:
# Combining the ratings from MSCI Refinitive and Yahoo finance 
def score(row):
    rating = 0
    if row['totalEsg'] != 0 & row['MSCI_Rating'] != 0 & row['Refinitive_Rating'] !=0: 
        rating = row['Refinitive_Rating']*0.4 + row['MSCI_Rating']*0.3 + row['totalEsg']*0.3
    elif row['totalEsg'] != 0 & row['Refinitive_Rating'] != 0:
        rating = row['Refinitive_Rating']*0.6 + row['totalEsg']*0.4
    elif row['totalEsg'] != 0 & row['MSCI_Rating'] != 0:
        rating = row['MSCI_Rating']*0.5 + row['totalEsg']*0.5
    elif row['MSCI_Rating'] != 0 & row['Refinitive_Rating'] != 0:
        rating = row['Refinitive_Rating']*0.6 + row['MSCI_Rating']*0.4
    elif row['Refinitive_Rating'] != 0:
        rating  = row['Refinitive_Rating']
    elif row['MSCI_Rating'] != 0:
        rating = row['MSCI_Rating']
    elif row['totalEsg'] != 0:
        rating = row['totalEsg']
    else:
        rating = 0
    return rating

In [ ]:
# Applying the scoring function to the dataset and creating a new column with the combined rating
df['ESG_rating'] = df.apply (lambda row: score(row), axis=1)

In [ ]:
# Dropping columns that are no longer relevant/required
df = df.drop(columns = ['Refinitive_Rating', 'MSCI_Rating', 'totalEsg', 'Unnamed: 0_y', 'Unnamed: 0_x', 'Ticker', 'ISIN', 'WBA Unique ID' , 'SEDOL Code', 'index', 'company_ticker','Unnamed: 0', 'peerGroup'])   

In [ ]:
df.to_excel('ESG_final_v3.xlsx') # exporting the database as an excel